In [72]:
import pandas as pd
import re

df = pd.read_csv("cleaned_costco_data.csv")

In [73]:
keywords = ["organic", "gluten free", "gluten-free", "GF", "vegan", "non-GMO", "non GMO", "kosher", "no added preservatives",  "no preservatives", "fat free", "fat-free", "low-fat", "low fat", "less fat", "reduced fat", "unsweetened", "no sugar added", "zero sugar", "sugar-free", "sugar free"]

In [74]:
def count_keywords(text1, text2, keywords):
    combined_text = f"{text1} {text2}".lower() if pd.notnull(text1) and pd.notnull(text2) else str(text1).lower() if pd.notnull(text1) else str(text2).lower()
    return {kw: int(bool(re.search(rf"\b{kw}\b", combined_text))) for kw in keywords}

# Apply function to create keyword presence columns
keyword_counts = df.apply(lambda row: count_keywords(row["Product Description"], row["Feature"], keywords), axis=1)

# Convert dictionary into a DataFrame
keyword_df = pd.DataFrame(keyword_counts.tolist())

# Merge with original data
df = pd.concat([df, keyword_df], axis=1)

In [75]:
# Define price thresholds using tertiles (33rd and 66th percentiles)
low_threshold = df["Price"].quantile(0.33)
high_threshold = df["Price"].quantile(0.66)

# Create a new column classifying products into three price categories
def classify_price(price):
    if price < low_threshold:
        return "Low-priced"
    elif price < high_threshold:
        return "Mid-priced"
    else:
        return "High-priced"

df["price_category"] = df["Price"].apply(classify_price)

In [76]:
keyword_groups = {
    "organic": ["organic"],
    "gluten-free": ["gluten free", "gluten-free", "GF"],
    "vegan": ["vegan"],
    "kosher": ["kosher"],
    "non-GMO": ["non-GMO", "non GMO"],
    "no-preservatives": ["no added preservatives", "no preservatives"],
    "low-fat": ["low-fat", "low fat", "reduced fat", "less fat", "fat free", "fat-free"],
    "sugar-free": ["unsweetened", "no sugar added", "zero sugar", "sugar-free", "sugar free"]
}

In [77]:
combined_keyword_counts = pd.DataFrame()

for new_col, old_cols in keyword_groups.items():
    combined_keyword_counts[new_col] = df[old_cols].sum(axis=1)

df_combined = pd.concat([df[["price_category"]], combined_keyword_counts], axis=1)

keyword_summary = df_combined.groupby("price_category").sum().reset_index()

print(keyword_summary)

  price_category  organic  gluten-free  vegan  kosher  non-GMO  \
0    High-priced       35           74     19      65        0   
1     Low-priced       68          108     23     223        0   
2     Mid-priced       65          106     22     166        0   

   no-preservatives  low-fat  sugar-free  
0                35        1           7  
1                19       25          35  
2                18       16          29  


In [78]:
df_combined.to_csv("updated_keywords_price_data.csv", index=False)